In [23]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline
import plotly.figure_factory as ff

from config import mapbox_api_key, b_api_key

import numpy as np
import pandas as pd

import requests

from pprint import pprint
from datetime import datetime as dt
from datetime import timedelta

In [2]:
# plotly.tools.set_credentials_file(username='ralfsantacruz', api_key='m43Oz35O5mKYCxDDJoDq')

In [3]:
def num_hours_ago(hours):
    """
    Returns UTC time x hours ago.
    """
#     from datetime import datetime as dt
    date = dt.utcnow() - timedelta(hours=hours)
    return dt.strftime(date, "%Y-%m-%dT%H:%M:%S")

In [4]:
def get_coords(location):
    """
    Returns coordinates of address for given input.
    """
    from config import g_api_key
    base_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={g_api_key}"
    
    data = requests.get(base_url).json()
    
    try:
        loc = data['results'][0]['geometry']['location']
        
        lat = loc['lat']
        lng = loc['lng']
    except:
        return "404"
    
    return lat,lng

In [5]:
def pollution(lat,lng):
    """
    Returns pollution data from coordinates.
    """
    from config import b_api_key 
    
    end_datetime = num_hours_ago(1)
    start_datetime = num_hours_ago(168)
    
    try:
        base_url = f"https://api.breezometer.com/air-quality/v2/historical/hourly?lat={lat}&lon={lng}&key={b_api_key}&start_datetime={start_datetime}&end_datetime={end_datetime}"
        data = requests.get(base_url).json()
        return data
    except:
        return None

In [6]:
def generate_plot(city):  
    '''
    Generates plot HTML for AQI for past week.
    '''
    try:    
        lat,lng = get_coords(city)
    except ValueError:
        return "Error: We could not find the coordinates for the entry you supplied."

    pollution_data = pollution(lat,lng)
    
    if pollution_data != None:
        try:
            x = [x['datetime'] for x in pollution_data['data'] if x['data_available'] == True]
            y = [y['indexes']['baqi']['aqi'] for y in pollution_data['data'] if y['data_available'] == True]

            text1 = [text['indexes']['baqi']['dominant_pollutant'] for text in pollution_data['data'] if text['data_available'] == True]
            text2 = [text['indexes']['baqi']['category'] for text in pollution_data['data'] if text['data_available'] == True]

            test = [f"Dominant pollutant: {x}\n{y}" for x,y in zip(text1,text2)]

            data = [go.Scattergl(x=x,
                               y=y, 
                               mode="markers", 
                               marker=dict(
                                size=12,
                                color = y, #set color equal to a variable
                                colorscale='Viridis',
                                showscale=True
                               ),  
                               name="AQI",
                                text= test)]

            layout = go.Layout(title=f"AQI for {city.title()}",
                            xaxis=dict(title='Date'),
                            yaxis=dict(title='AQI'))

            fig = go.Figure(data=data, layout=layout)
            
            return py.plot(fig, filename='jupyter-basic_line',auto_open=False)
        
        except:
            return "404"        

# Calling functions to plot:

In [7]:
# x,y = get_coords("beijing")

In [8]:
# print(x)
# pollution_data = pollution(x,y)

In [9]:
# generate_plot("san francisco")

# x = [x['datetime'] for x in pollution_data['data'] if x['data_available'] == True]
# y = [y['indexes']['baqi']['aqi'] for y in pollution_data['data'] if y['data_available'] == True]
# text1 = [text['indexes']['baqi']['dominant_pollutant'] for text in pollution_data['data'] if text['data_available'] == True]
# text2 = [text['indexes']['baqi']['category'] for text in pollution_data['data'] if text['data_available'] == True]

# test = [f"Dominant pollutant: {x}\n{y}" for x,y in zip(text1,text2)]

# data = [go.Scattergl(x=x,
#                    y=y, 
#                    mode="markers", 
#                    marker=dict(
#                     size=10,
#                     color = y, #set color equal to a variable
#                     colorscale='Viridis',
#                     showscale=True,
#                     reversescale=True
#                    ),  
#                    name="AQI",
#                     text= test)]

# layout = go.Layout(title="AQI for beijing",
#                 xaxis=dict(title='Date'),
#                 yaxis=dict(title='AQI'))

# fig = go.Figure(data=data, layout=layout)

# # py.iplot(fig, filename='jupyter-basic_line')

# p = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

In [10]:
# print(p)
# print(text1)
# print(text2)
# print(test)
# x,y = "404"
# print(p)

In [15]:

# df_wind = pd.read_csv('https://plot.ly/~datasets/2805.csv')

# df_known_capacity = df_wind[ df_wind['total_cpcy'] != -99999.000 ]
# df_sum = df_known_capacity.groupby('manufac')['total_cpcy'].sum().sort_values(ascending=False).to_frame()

# df_farms = pd.read_csv('https://plot.ly/~jackp/17256.csv')
# df_farms.set_index('Wind Farm', inplace=True)

# wind_farms=list([
#     dict(
#         args=[ { 
#             'mapbox.center.lat':38,
#             'mapbox.center.lon':-94,
#             'mapbox.zoom':3,
#             'annotations[0].text':'All US wind turbines (scroll to zoom)'
#         } ],
#         label='USA',
#         method='relayout'
#     )
# ])

# for farm, row in df_farms.iterrows():
#     desc = []
#     for col in df_farms.columns:
#         if col not in ['DegMinSec','Latitude','Longitude']:
#             if str(row[col]) not in ['None','nan','']: 
#                 desc.append( col + ': ' + str(row[col]).strip("'") )
#     desc.insert(0, farm)
#     wind_farms.append( 
#         dict(
#             args=[ { 
#                 'mapbox.center.lat':row['Latitude'], 
#                 'mapbox.center.lon':float(str(row['Longitude']).strip("'")), 
#                 'mapbox.zoom':9,
#                 'annotations[0].text': '<br>'.join(desc)
#             } ],
#             label=' '.join(farm.split(' ')[0:2]),
#             method='relayout'
#         )
#     )

# data = []
# for mfr in list(df_sum.index):
#     if mfr != 'unknown':
#         trace = dict(
#             lat = df_wind[ df_wind['manufac'] == mfr ]['lat_DD'],
#             lon = df_wind[ df_wind['manufac'] == mfr ]['long_DD'],
#             name = mfr,
#             marker = dict(size = 4),
#             type = 'scattermapbox'
#         )
#     data.append(trace)

# # mapbox_access_token = 'insert mapbox token here'

# layout = dict(
#     height = 800,
#     margin = dict( t=0, b=0, l=0, r=0 ),
#     font = dict( color='#FFFFFF', size=11 ),
#     paper_bgcolor = '#000000',
#     mapbox=dict(
#         accesstoken=mapbox_api_key,
#         bearing=0,
#         center=dict(
#             lat=38,
#             lon=-94
#         ),
#         pitch=0,
#         zoom=3,
#         style='dark'
#     ),
# )

# updatemenus=list([
#     dict(
#         buttons = wind_farms[0:10],
#         pad = {'r': 0, 't': 10},
#         x = 0.1,
#         xanchor = 'left',
#         y = 1.0,
#         yanchor = 'top',
#         bgcolor = '#AAAAAA',
#         active = 99,
#         bordercolor = '#FFFFFF',
#         font = dict(size=11, color='#000000')
#     ),
#     dict(
#         buttons=list([
#             dict(
#                 args=['mapbox.style', 'dark'],
#                 label='Dark',
#                 method='relayout'
#             ),                    
#             dict(
#                 args=['mapbox.style', 'light'],
#                 label='Light',
#                 method='relayout'
#             ),
#             dict(
#                 args=['mapbox.style', 'satellite'],
#                 label='Satellite',
#                 method='relayout'
#             ),
#             dict(
#                 args=['mapbox.style', 'satellite-streets'],
#                 label='Satellite with Streets',
#                 method='relayout'
#             )                    
#         ]),
#         direction = 'up',
#         x = 0.75,
#         xanchor = 'left',
#         y = 0.05,
#         yanchor = 'bottom',
#         bgcolor = '#000000',
#         bordercolor = '#FFFFFF',
#         font = dict(size=11)
#     ),        
# ])

# annotations = list([
#     dict(text='All US wind turbines (scroll to zoom)', font=dict(color='magenta',size=14), borderpad=10, 
#          x=0.05, y=0.05, xref='paper', yref='paper', align='left', showarrow=False, bgcolor='black'),
#     dict(text='Wind<br>Farms', x=0.01, y=0.99, yref='paper', align='left', showarrow=False,font=dict(size=14))
# ])

# layout['updatemenus'] = updatemenus
# layout['annotations'] = annotations

# figure = dict(data=data, layout=layout)
# py.iplot(figure, filename='wind-turbine-territory-dropdown')

In [68]:
def pollution_epa(lat,lng):
    """
    Returns EPA pollution data from coordinates.
    """
    
    base_url = f"https://api.breezometer.com/air-quality/v2/current-conditions?lat={lat}&lon={lng}&key={b_api_key}&features=local_aqi"
    data = requests.get(base_url).json()
    return data

In [24]:
df = pd.read_csv('california_cities.csv')

In [71]:
def append_geodata(df):    
    """
    Appends pollution data to city list dataframe.
    """
    
    aqi = []
    category = []
    dominant_pollutant = []


    for lat,lng in zip(df['lat'],df['lng']):
        data = pollution_epa(lat,lng)
        index = data['data']['indexes']['usa_epa']
        
        air_quality = index['aqi']
        categories = index['category']
        dom_pollutant = index['dominant_pollutant']
        
        
        aqi.append(air_quality)
        category.append(categories)
        dominant_pollutant.append(dom_pollutant)
        
    df['aqi'] = aqi
    df['category'] = category
    df['dominant_pollutant'] = dominant_pollutant
    
    return df
#         print(aqi)
#         print(category)
#         print(dominant_pollutant)
    

In [72]:
df = append_geodata(df)

In [100]:
# df.to_csv('california_pollution.csv',index=False)
df.head()

,cities,lat,lng,aqi,category,dominant_pollutant
0,Adelanto,34.582770,-117.409215,41,Good air quality,o3
1,Agoura Hills,34.153340,-118.761676,48,Good air quality,pm25
2,Alameda,37.765206,-122.241636,61,Moderate air quality,pm25
3,Albany,37.886870,-122.297748,61,Moderate air quality,pm25
4,Alhambra,34.095287,-118.127015,66,Moderate air quality,pm25


In [89]:
def marker_text(df):
    text = []
    for row in df.itertuples():
        a = getattr(row, "cities")
        b = getattr(row, "category")
        c = getattr(row, "aqi")
        d = getattr(row, "dominant_pollutant")
        string = f"{a}<br>{b}<br>AQI: {c}<br>Dominant pollutant: {d}"
        text.append(string)
    return text

In [101]:
data = [
    go.Scattermapbox(
        lat=round(df['lat'],3),
        lon=round(df['lng'],3),
        mode='markers',
        marker=dict(
            size=df['aqi']/10,
            color= df['aqi'],
            colorscale = 'Jet',
        ),
        text= marker_text(df)
    )
]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_api_key,
        bearing=0,
        center=dict(
            lat=36,
            lon=-119
        ),
        style='dark',
        pitch=0,
        zoom=4
    ),
)

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='CA Mapbox')
py.plot(fig, filename='CA Mapbox',auto_open=False)
# cali_plot = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

C:\Users\Rafael\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



'https://plot.ly/~ralfsantacruz/12'

In [94]:
print(cali_plot)

<div><div id="27ed2a78-1c2f-413d-be6e-9e56245889f1" style="height: 100%; width: 100%;" class="plotly-graph-div"></div><script type="text/javascript">window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot("27ed2a78-1c2f-413d-be6e-9e56245889f1", [{"lat": [34.583, 34.153, 37.765, 37.887, 34.095, 33.568, 41.487, 38.419, 38.175, 33.837, 40.448, 38.068, 38.005, 34.501, 34.14, 40.867, 35.119, 33.866, 35.202, 35.489, 37.461, 37.348, 38.897, 33.343, 36.004, 34.134, 35.373, 34.085, 33.926, 34.896, 33.929, 33.978, 33.965, 33.882, 37.52, 37.873, 38.049, 37.872, 34.074, 34.244, 39.412, 37.361, 40.883, 33.618, 34.147, 32.979, 33.916, 37.932, 37.681, 34.614, 33.867, 34.181, 37.578, 34.137, 32.679, 35.126, 34.004, 33.126, 38.579, 34.216, 37.287, 33.685, 36.975, 33.158, 36.555, 34.399, 33.832, 33.781, 37.595, 33.858, 39.728, 34.012, 33.99, 37.123, 32.64, 38.707, 34.097, 37.941, 38.958, 38.805, 36.825, 33.68, 36.14, 39.101, 37.677, 34.074, 39.214, 34.001, 33.8